# Project: Data Warehouse

### Introduction

A music streaming startup, Sparkify, has grown their user base and song database and want to move their processes and data onto the cloud. Their data resides in S3, in a directory of JSON logs on user activity on the app, as well as a directory with JSON metadata on the songs in their app.

As their data engineer, you are tasked with building an ETL pipeline that extracts their data from S3, stages them in Redshift, and transforms data into a set of dimensional tables for their analytics team to continue finding insights in what songs their users are listening to. You'll be able to test your database and ETL pipeline by running queries given to you by the analytics team from Sparkify and compare your results with their expected results.


### Project Description

In this project, you'll apply what you've learned on data warehouses and AWS to build an ETL pipeline for a database hosted on Redshift. To complete the project, you will need to load data from S3 to staging tables on Redshift and execute SQL statements that create the analytics tables from these staging tables.

Here are the steps you will take to create the schema for your tables and then copy teh data from Amazon S3 to Amazon Redshift. 

### Step 0: Create an Amazon Redshift Table

Lets start with creating an Amazon Redshift table using the AWS CLI. You will need the AWS CLI installed for this to work and appropriate IAM permissions. If you are not sure what permissions you have, you can check your IAM user if you have IAM permissions or atleast type `aws configure` in the command line to check whether you have an access key and secret key.

We are going to be using the cheapest Redshift cluster we could find. Through the docs/pricing page, `dc2.large` is the smallest and the cheapest that includes the query option.

**Note**: If you can't access the cluster `connection refused` error then do the following
1. click on your cluster name
2. open the 'VPC security group' that your cluster belongs to
3. At the bottom of the screen, click **Inbound**
4. Click **Edit** > **Add new rule** and add a new **Redshift** rule with source **My IP**. DO NOT OPEN THE SG to all IP



In [1]:
import boto3

iam = boto3.client('iam')
redshifts3_arn = iam.get_role(RoleName='RedshiftS3')['Role']['Arn']



In [2]:
#%%bash -s "$redshifts3_arn"
#
#aws redshift create-cluster --node-type dc2.large \
#                            --cluster-type single-node \
#                            --master-username admin \
#                            --master-user-password TopSecret1 \
#                            --cluster-identifier mycluster1 \
#                            --db-name dev \
#                            --iam-roles $1

### Step 1: Add all the information from your cluster into variables

You can find most information above when you create the cluster. For the `host` name, check the redshift cluster in the AWS console

In [3]:
#[CLUSTER]
HOST= 'mycluster1.cyqgskzvawwc.us-west-2.redshift.amazonaws.com'
DB_NAME= 'dev'
DB_USER= 'admin'
DB_PASSWORD= 'TopSecret1'
DB_PORT= '5439'

#[IAM_ROLE]
ARN=redshifts3_arn

#[S3]
LOG_DATA='s3://udacity-dend/log_data'
LOG_JSONPATH='s3://udacity-dend/log_json_path.json'
SONG_DATA='s3://udacity-dend/song_data'

### Step 2: Design schemas for your fact and dimension tables.

We will create multiple dimension tables and single fact table. The schema for the tables is given below:

- Fact Table: `songplays` - records in event data associated with song plays i.e. records with page NextSong with variables `songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent`
    
- Dimension Tables: There are 4 dimension tables `users` table, `songs` table, `artists` table and `time` table.
    - users - users in the app. Variables: `user_id, first_name, last_name, gender, level`.
    - songs - songs in music database. Variables: `song_id, title, artist_id, year, duration`
    - artists - artists in music database. Variables: `artist_id, name, location, lattitude, longitude`
    - time - timestamps of records in songplays broken down into specific units. Variables: `start_time, hour, day, week, month, year, weekday`


In [85]:
import configparser

# DROP TABLES

staging_events_table_drop = "DROP TABLE IF EXISTS staging_events"
staging_songs_table_drop = "DROP TABLE IF EXISTS staging_songs"
songplay_table_drop = "DROP TABLE IF EXISTS songplays"
user_table_drop = "DROP TABLE IF EXISTS users"
song_table_drop = "DROP TABLE IF EXISTS songs"
artist_table_drop = "DROP TABLE IF EXISTS artists"
time_table_drop = "DROP TABLE IF EXISTS time"

# CREATE TABLES


staging_events_table_create= ("""

CREATE TABLE staging_events
(
    artist            VARCHAR(255),
    auth              VARCHAR(100),
    firstName         TEXT,
    gender            TEXT,
    itemInSession     INTEGER,
    lastName          TEXT,
    length            FLOAT DEFAULT 0,
    level             TEXT,
    location          VARCHAR(50),
    method            VARCHAR(50),
    page              VARCHAR(50),
    registration      REAL,
    sessionId         INTEGER NOT NULL,
    song              VARCHAR(255),
    status            INTEGER,
    ts                BIGINT,
    userAgent         VARCHAR(255),
    userId            INTEGER
)

""")

staging_songs_table_create = ("""

CREATE TABLE staging_songs 
(
  artist_id           TEXT,
  artist_latitude     FLOAT,
  artist_location     VARCHAR(255),
  artist_longitude    FLOAT,
  artist_name         VARCHAR(255),
  duration            FLOAT,
  num_songs           INTEGER,
  song_id             VARCHAR(50),
  title               VARCHAR(255),
  year                INTEGER
);

""")

songplay_table_create = ("""
CREATE TABLE IF NOT EXISTS songplays (
                                songplay_id bigint IDENTITY(0,1),
                                start_time TIMESTAMP, 
                                user_id INT, 
                                level VARCHAR, 
                                song_id VARCHAR(64), 
                                artist_id VARCHAR(64), 
                                session_id INT, 
                                location TEXT, 
                                user_agent TEXT
                                )
""")

user_table_create = ("""
CREATE TABLE IF NOT EXISTS users (
                                user_id INT PRIMARY KEY, 
                                first_name VARCHAR(128), 
                                last_name VARCHAR(128), 
                                gender VARCHAR(5), 
                                level VARCHAR(5)
                                )
""")

song_table_create = ("""
CREATE TABLE IF NOT EXISTS songs (
                                song_id varchar PRIMARY KEY, 
                                title TEXT, 
                                artist_id VARCHAR(64), 
                                year INT, 
                                duration FLOAT(4)
                                )
""")

artist_table_create = ("""
CREATE TABLE IF NOT EXISTS artists (
                                artist_id varchar(64) PRIMARY KEY, 
                                name VARCHAR(255), 
                                location VARCHAR(255), 
                                lattitude FLOAT, 
                                longitude FLOAT
                                )
""")

time_table_create = ("""
CREATE TABLE IF NOT EXISTS time (
                                start_time TIMESTAMP primary key, 
                                hour INT, 
                                day INT, 
                                week INT, 
                                month INT, 
                                year INT, 
                                weekday TEXT
                                )
""")

### Step 3: Copying the raw Json data into a staging table in Redshift

You will start with the raw json files in the s3 bucket `udacity-dend` and you will copy the data into the respective redshift tables `staging_events` and `staging_songs`. Once you are done with copying the data, you will use this data to insert the data into the respective dimension table

In [ ]:
# STAGING TABLES

staging_events_copy = ("""
copy staging_events
from 's3://udacity-dend/log_data'
iam_role '{}'
FORMAT AS json '{}'
""").format(ARN,LOG_JSONPATH)

staging_songs_copy = ("""
copy staging_songs
from 's3://udacity-dend/song_data'
iam_role '{}'
BLANKSASNULL
EMPTYASNULL
FORMAT AS json 'auto'
""").format(ARN)

# FINAL TABLES

songplay_table_insert = ("""
INSERT into songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
(select 
  TIMESTAMP 'epoch' + e.ts/1000 * interval '1 second', 
  e.userId, 
  e.level, 
  (select s.song_id from staging_songs as s where s.title = e.song and s.artist_name = e.artist) as song_id,
  (select s.artist_id from staging_songs as s where s.title = e.song and s.artist_name = e.artist) as artist_id,
  e.sessionId,
  e.location,
  e.userAgent
from staging_events as e
where e.page = 'NextSong');
""")

user_table_insert = ("""
INSERT into users
(select DISTINCT
  userId,
  firstName,
  lastName,
  gender,
  level
FROM staging_events
WHERE page = 'NextSong'
)
""")

song_table_insert = ("""
INSERT into songs 
(SELECT DISTINCT
    song_id
    title,
    artist_id,
    year,
    duration
FROM staging_songs

)
""")

artist_table_insert = ("""
INSERT into artists 
(SELECT DISTINCT
    artist_id,
    artist_name,
    artist_location,
    artist_latitude,
    artist_longitude
FROM staging_songs
)
""")

time_table_insert = ("""
INSERT into time 
(Select 
    distinct t_start_time,
    EXTRACT(HOUR FROM t_start_time) As t_hour,
    EXTRACT(DAY FROM t_start_time) As t_day,
    EXTRACT(WEEK FROM t_start_time) As t_week,
    EXTRACT(MONTH FROM t_start_time) As t_month,
    EXTRACT(YEAR FROM t_start_time) As t_year,
    EXTRACT(DOW FROM t_start_time) As t_weekday
FROM (
SELECT distinct TIMESTAMP 'epoch' + ts/1000 * interval '1 second' as t_start_time
FROM staging_events 
WHERE page = 'NextSong'
)
)
""")

# QUERY LISTS

create_table_queries = [staging_events_table_create, staging_songs_table_create, songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]
drop_table_queries = [staging_events_table_drop, staging_songs_table_drop, songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]
copy_table_queries = [staging_events_copy, staging_songs_copy]
insert_table_queries = [songplay_table_insert, user_table_insert, song_table_insert, artist_table_insert, time_table_insert]


### Step 4: Create the tables in the redshfit cluster (Create_tables.py)

Now you will connect to the redshift cluster and you will use the queries above to create the tables in the redshift cluster

In [66]:
#import configparser
import psycopg2
#from sql_queries import create_table_queries, drop_table_queries


def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables_main():
    #config = configparser.ConfigParser()
    #config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(HOST, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT))
    cur = conn.cursor()

    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


create_tables_main()

### Step 4: Create the tables in the redshfit cluster (etl.py)

Now you will connect to the redshift cluster and you will use the queries above to insert the rows from the staging tables to the fact and dimension tables. You can use another connector to connect to the cluster. This might help in debugging error 

In [67]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(HOST, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT))
cur = conn.cursor()

In [68]:
#import configparser
import psycopg2
#from sql_queries import copy_table_queries, insert_table_queries


def load_staging_tables(cur, conn):
    for i,query in enumerate(copy_table_queries):
        print(f'Query {i+1}/{len(copy_table_queries)} for staging tables')
        cur.execute(query)
        conn.commit()
        print('Done')


def insert_tables(cur, conn):
    for i,query in enumerate(insert_table_queries):
        print(f'Query {i+1}/{len(insert_table_queries)} for inserting tables')
        cur.execute(query)
        conn.commit()
        print('Done')

In [69]:
load_staging_tables(cur, conn)

Query 1/2 for staging tables
Done
Query 2/2 for staging tables
Done


In [86]:
#conn.rollback()
insert_tables(cur, conn)

Query 1/5 for inserting tables
Done
Query 2/5 for inserting tables
Done
Query 3/5 for inserting tables
Done
Query 4/5 for inserting tables
Done
Query 5/5 for inserting tables
Done


In [87]:
conn.close()